# Kalkulieren der Key Performance Indicator zu einen Produktionsplan

In [1]:
import pandas as pd
from datetime import timedelta

In [ ]:
df = pd.read_excel("../data/test.xlsx")

1. Gesamtrüstzeit während einer Periode

In [ ]:
def overall_setup_time(df, col_production_date, col_machine, col_setup):
    """Calculates the overall setup time per machine per day for a given production plan.

    Parameters
    ----------
    df : pd.DataFrame
        DataFrame with orders
    col_production_date : string
        Name of the column with the planned production date of an order
    col_machine : string
        Name of the column in df with the planned machine for production of an order
    col_setup : string
        Name of the column with the planned setup time for the order

    Returns
    -------
    pd.DataFrame
        DataFrame with the setup times per machine per day
    """
    dates = df[col_production_date].unique() # per date
    machines = df[col_machine].unique() # per machine
    df_setup_times = pd.DataFrame(columns=['machine_id', 'date', 'setup_time']) # empty df
    for date in dates: # Iterate over dates and machines
        for machine in machines:
            df_day_machine = df[[df[col_production_date]==date and df[col_machine]==machine]]
            overall_setup = df_day_machine[col_setup].sum()
            new_row = {'machine_id': machine,
                        'date': date, 'setup_time': overall_setup}
            df_setup_times = df_setup_times.append(new_row, ignore_index=True)
    return df_setup_times

df_setup_times = overall_setup_time(df, 'Production_date', 'machine_id', 'Ruestzeit_Soll')
df_setup_times

2. Anzahl verspäteter Aufträge

In [ ]:
def num_tardy_jobs(df,col_production_date, col_delivery_date):
    """Calculates the number of tardy jobs.

    Parameters
    ----------
    df : pd.DataFrame
        DataFrame with orders and defined columns
    col_production_date : string
        Name of the column with the planned production date of an order
    col_delivery_date : string
        Name of the column with the delivery date of an order

    Returns
    -------
    int
        Number of tardy jobs
    """
    tardy_jobs = df[[df[col_production_date]>= df[col_delivery_date]]].shape[0]
    return tardy_jobs

tardy_jobs = num_tardy_jobs(df, 'Production_date', 'LTermin')
tardy_jobs

3. Efficiency: 
Definiert als Gesamtproduktionszeit/(Gesamtrüstzeit + Gesamtproduktionszeit)

In [ ]:
def calculate_efficiency(df, col_production_date, col_machine, col_setup, col_runtime):
    """"Calculates the efficiency of a production plan per machine per date.
    Efficiency is defined as *overall production time / (overall setup time + overall production time)*.

    Parameters
    ----------
    df : pd.DataFrame 
        DataFrame with production orders and the defined columns
    col_production_date : string
        Name of the column with the planned production date of an order
    col_machine : string
        Name of the column in df with the planned machine for production of an order
    col_setup : string
        Name of the column with the planned setup time for the order
    col_runtime : sting
        Name of the column with the planned runtime for an order

    Returns
    -------
    pd.DataFrame
        df with efficiency of the production plan per day per machine
    """
    # Per day per machine
    dates = df[col_production_date].unique()
    machines = df[col_machine].unique()
    df_efficiency = pd.DataFrame(columns=['machine_id', 'date', 'efficiency'])
    for date in dates:
        for machine in machines:
            # Select machine and day in df and calculate efficiency
            df_day_machine = df[[df[col_production_date]==date and df[col_machine]==machine]]
            overall_setup = df_day_machine[col_setup].sum()
            overall_runtime = df_day_machine[col_runtime].sum()
            efficiency = overall_runtime/(overall_setup + overall_runtime)
            new_row = {'machine_id': machine,
                        'date': date, 'efficiency': efficiency}
            df_efficiency = df_efficiency.append(new_row, ignore_index=True)
    return df_efficiency

df_efficiency = calculate_efficiency(df, 'Production_date', 'machine_id', 'Ruestzeit_Soll','Laufzeit_Soll')
df_efficiency